In [5]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from simple_pe.waveforms import parameter_bounds, check_physical

In [2]:
with open(os.path.join('data', '4d_samples_pe_result'), 'rb') as fp:
    pe_result = pickle.load(fp)
metric = pe_result.metric

In [3]:
def const_mm_point(metric, keys, mm, target_par, base_vals, direction=1):

    # Find A
    target_ind = keys.index(target_par)
    A = metric[target_ind][target_ind]

    # Find B
    param_keys = [key for key in keys if key != target_par]
    B = np.array([metric[target_ind][k] for k in range(len(keys)) if k != target_ind])

    # Find inverse of C
    C = np.delete(metric, (target_ind), axis=0)
    C = np.delete(C, (target_ind), axis=1)
    C_inv = np.linalg.inv(C)

    # Find change in target parameter to reach mismatch
    BC_term = -np.matmul(B, np.matmul(C_inv, B))
    d_target = direction*np.sqrt(mm/(A+BC_term))

    # Find extreme point at specified mismatch
    dxs = -1*np.matmul(C_inv, B) * d_target
    extreme_point = {target_par: base_vals[target_par]+d_target}
    for i, key in enumerate(param_keys):
        extreme_point[key] = dxs[i] + base_vals[key]

    return extreme_point

In [46]:
def degeneracy_line(metric, e10s, dirs=['ecc10sqrd', 'chirp_mass', 'symmetric_mass_ratio', 'chi_eff'], mins=None, maxs=None):

    # Setup
    if mins is None:
        mins = parameter_bounds.param_mins
    if maxs is None:
        maxs = parameter_bounds.param_maxs
    base_vals = {direct: metric.x[direct][0] for direct in dirs}
    e10s = np.array([e10s]).flatten()
    rail_dirs = dirs.copy()
    rail_dirs.remove('ecc10sqrd')
    railing = []
    for i in range(len(e10s)):
        railing.append('0'*(len(dirs)-1))

    # Loop over iterations
    finished = np.zeros_like(e10s)
    degen_points = {}
    new_vals = [base_vals for i in range(len(e10s))]
    while np.sum(finished) < len(finished):

        # Calculate degenerate directions for all necessary projections
        for degen_key in set(railing):
            if degen_key not in degen_points.keys():
                proj_dirs = [rail_dirs[i] for i in range(len(rail_dirs)) if int(degen_key[i])]
                keep_dirs = [key for key in metric.dxs.keys() if key not in proj_dirs]
                metric.project_metric(keep_dirs)
                degen_points[degen_key] = const_mm_point(metric.projected_metric, keep_dirs, metric.projected_mismatch, 'ecc10sqrd', base_vals)
                for direct in proj_dirs:
                    degen_points[degen_key][direct] = base_vals[direct]

        # For each eccentricity value
        for i, e10 in enumerate(e10s):
            if finished[i]:
                continue

            # Calculate new value
            dist = (e10**2 - new_vals[i]['ecc10sqrd'])/(degen_points[railing[i]]['ecc10sqrd'] - base_vals['ecc10sqrd'])
            dx = {}
            for direct in dirs:
                dx[direct] = dist*(degen_points[railing[i]][direct] - base_vals[direct])
            new_vals_pos, dx_pos = new_vals[i].copy(), dx.copy()
            for key in list(dx_pos.keys()):
                if dx_pos[key] == 0:
                    new_vals_pos.pop(key)
                    dx_pos.pop(key)
            alpha = check_physical(new_vals_pos, dx_pos, 1, maxs=maxs, mins=mins)
            new_point = {}
            for direct in dirs:
                new_point[direct] = alpha*dist*(degen_points[railing[i]][direct] - base_vals[direct]) + new_vals[i][direct]
            new_vals[i] = new_point

            # Check if railing
            rail = ''
            if alpha < 1:
                for j, raildir in enumerate(rail_dirs):
                    if np.isclose(new_point[raildir], maxs[raildir], atol=1e-15) or np.isclose(new_point[raildir], mins[raildir], atol=1e-15):
                        rail += '1'
                    else:
                        rail += '0'
                if rail == railing[i]:
                    finished[i] = True
                else:
                    railing[i] = rail
            else:
                finished[i] = True

    # Return as array or single value
    new_vals = np.array(new_vals)
    if len(np.shape(new_vals.squeeze())) == 0:
        new_vals = new_vals[0]

    return new_vals

In [48]:
mins = parameter_bounds.param_mins.copy()
maxs = parameter_bounds.param_maxs.copy()
mins['symmetric_mass_ratio'] = 0.20
degeneracy_line(metric, np.linspace(0, 1, 6), mins=mins, maxs=maxs)

array([{'ecc10sqrd': 0.0, 'chirp_mass': 24.70401677956892, 'symmetric_mass_ratio': 0.2061104690205152, 'chi_eff': 0.13994519741392042},
       {'ecc10sqrd': 0.04000000000000001, 'chirp_mass': 24.177888399169536, 'symmetric_mass_ratio': 0.2022766566716489, 'chi_eff': 0.11428548356502749},
       {'ecc10sqrd': 0.16000000000000003, 'chirp_mass': 22.599503257971378, 'symmetric_mass_ratio': 0.2, 'chi_eff': 0.03730634201834722},
       {'ecc10sqrd': 0.3600000000000001, 'chirp_mass': 19.968861355974447, 'symmetric_mass_ratio': 0.2, 'chi_eff': -0.09099222722612052},
       {'ecc10sqrd': 0.6400000000000001, 'chirp_mass': 16.28596269317874, 'symmetric_mass_ratio': 0.2, 'chi_eff': -0.2706102241683754},
       {'ecc10sqrd': 1.0, 'chirp_mass': 11.550807269584269, 'symmetric_mass_ratio': 0.2, 'chi_eff': -0.5015476488084172}],
      dtype=object)

In [49]:
degeneracy_line(metric, np.linspace(0, 1, 6))

array([{'ecc10sqrd': 0.0, 'chirp_mass': 24.70401677956892, 'symmetric_mass_ratio': 0.2061104690205152, 'chi_eff': 0.13994519741392042},
       {'ecc10sqrd': 0.04000000000000001, 'chirp_mass': 24.177888399169536, 'symmetric_mass_ratio': 0.2022766566716489, 'chi_eff': 0.11428548356502749},
       {'ecc10sqrd': 0.16000000000000003, 'chirp_mass': 22.599503257971374, 'symmetric_mass_ratio': 0.1907752196250501, 'chi_eff': 0.03730634201834873},
       {'ecc10sqrd': 0.3600000000000001, 'chirp_mass': 19.968861355974443, 'symmetric_mass_ratio': 0.17160615788071876, 'chi_eff': -0.09099222722611595},
       {'ecc10sqrd': 0.6400000000000001, 'chirp_mass': 16.28596269317874, 'symmetric_mass_ratio': 0.14476947143865487, 'chi_eff': -0.2706102241683664},
       {'ecc10sqrd': 0.9999999999999999, 'chirp_mass': 11.550807269584263, 'symmetric_mass_ratio': 0.11026516029885848, 'chi_eff': -0.5015476488084024}],
      dtype=object)